### IMPORTS

In [1]:
import ipywidgets
import cv2
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
import traitlets
import time
import shutil
import os
import pandas as pd

<br>

### HYPERPARAMETERS

In [24]:
STEERING_OFFSET = 0.05
THROTTLE_GAIN = 0.7

WIDTH = 224
HEIGHT = 224
FPS = 10
SHOW_CAMERA_PREVIEW = True

DATASETS_DIR = "/home/greg/datasets/"
TMP_DATASET_DIR = DATASETS_DIR + "tmp/"
ANNOTATIONS_FILE = "annotations.csv"
TMP_ANNOTATIONS = TMP_DATASET_DIR + ANNOTATIONS_FILE
MAIN_DATASET_DIR = DATASETS_DIR + "camera-basic_motion_0_training/"
MAIN_ANNOTATIONS = MAIN_DATASET_DIR + ANNOTATIONS_FILE

<br>

### DATA COLLECTION

In [25]:
def reset_temp_dataset_dir():
    if not os.path.exists(TMP_DATASET_DIR):
        os.makedirs(TMP_DATASET_DIR)
    else:
        for file in os.listdir(TMP_DATASET_DIR):
            os.remove(TMP_DATASET_DIR+file)

reset_temp_dataset_dir()

if not os.path.exists(MAIN_DATASET_DIR):
    os.makedirs(MAIN_DATASET_DIR)

In [26]:
def start_recording():
    reset_temp_dataset_dir()
    
def save_recording():
    for file in os.listdir(TMP_DATASET_DIR):
        if file.endswith('.csv'):
            if os.path.exists(MAIN_ANNOTATIONS) and os.stat(MAIN_ANNOTATIONS).st_size > 0:
                with open(MAIN_ANNOTATIONS, 'a') as main:
                    with open(TMP_ANNOTATIONS) as tmp:
                        for line in tmp:
                            main.write(line)
                        tmp.close()
                    main.close()
                continue
        shutil.move(TMP_DATASET_DIR+file, MAIN_DATASET_DIR+file)
    
    reset_temp_dataset_dir()

<br>

### CAR CONTROLLER

In [5]:
car = NvidiaRacecar()
car.throttle_gain = THROTTLE_GAIN
car.steering_offset = STEERING_OFFSET

In [6]:
controller = ipywidgets.widgets.Controller(index=0)
display(controller)
time.sleep(1.0)

Controller()

In [8]:
# Specific to Gamepad Mode 2

steering_link = traitlets.dlink((controller.axes[2], 'value'), (car, 'steering'), transform=lambda x: -x)
throttle_link = traitlets.dlink((controller.axes[1], 'value'), (car, 'throttle'), transform=lambda x: x)

full_throttle_link = traitlets.dlink((controller.buttons[6], 'value'), (car, 'throttle'), transform=lambda x: -1.0 if x > 0.5 else 0)
half_throttle_link = traitlets.dlink((controller.buttons[4], 'value'), (car, 'throttle'), transform=lambda x: -0.5 if x > 0.5 else 0)
brake_throttle_link = traitlets.dlink((controller.buttons[7], 'value'), (car, 'throttle'), transform=lambda x: 0.0)
half_treverse_link = traitlets.dlink((controller.buttons[5], 'value'), (car, 'throttle'), transform=lambda x: 0.5 if x > 0.5 else 0)

def is_valid_press(x):
    return x['name'] == 'pressed' and x['new']

controller.buttons[2].observe(lambda x: start_recording() if is_valid_press(x) else ())
controller.buttons[1].observe(lambda x: save_recording() if is_valid_press(x) else ())

<br>

### CAMERA

In [9]:
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=FPS)
camera.running = True

In [27]:
def save_annotated_camera_frame(frame):
    timestamp = str(int(time.monotonic()*1e9))
    encoded_image = cv2.imencode('.jpg', frame)[1]
    
    def save_camera_frame():
        decoded_image = cv2.imdecode(encoded_image,cv2.IMREAD_COLOR)
        cv2.imwrite(TMP_DATASET_DIR+timestamp+".jpg", decoded_image)
        
    def save_annotation():
        with open(TMP_ANNOTATIONS, 'a') as f: 
            f.write(timestamp + ", " + str(round(car.steering, 2)) + ", " + str(round(car.throttle, 2)) + "\n")
            f.close()
    
    save_camera_frame()
    save_annotation()

    return bytes(encoded_image)

In [11]:
image = ipywidgets.Image(format='jpeg')
if SHOW_CAMERA_PREVIEW:
    display(image)

Image(value=b'', format='jpeg')

In [12]:
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=save_annotated_camera_frame)